Importing Files

In [18]:
import cv2
import os
import numpy

Run following block to input user faces to system.
(give folder path as data/at and create a user name folder (ex: kisal) for each user)

In [19]:
import cv2
import os

output_folder = "data/at/kisal"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


camera = cv2.VideoCapture(0)
count = 0
while cv2.waitKey(1) == -1 :
    success, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize = (120,120))
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2 )
        face_im = cv2.resize(gray[y:y+h, x:x+w], (200,200) )
        face_filename = '%s/%d.pgm' % (output_folder, count)
        cv2.imwrite(face_filename, face_im)
        count +=1
    cv2.imshow('CapturingFaces',frame)
camera.release()
cv2.destroyWindow('CapturingFaces')

Run following cell

In [20]:
def read_images(path, image_size):
    names = []
    training_images, training_labels = [], []
    label = 0
    for dirname, subdirnames, filenames in os.walk(path):
        for subdirname in subdirnames:
            names.append(subdirname)
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                img = cv2.imread(os.path.join(subject_path, filename),  cv2.IMREAD_GRAYSCALE)
                if img is None:
                    # The file cannot be loaded as an image.
                    # Skip it.
                    continue
                img = cv2.resize(img, image_size)
                training_images.append(img)
                training_labels.append(label)
            label += 1
    training_images = numpy.asarray(training_images, numpy.uint8)
    training_labels = numpy.asarray(training_labels, numpy.int32)
    return names, training_images, training_labels

Train the model running following block

In [21]:
path_to_training_images = 'data/at'
training_image_size = (200, 200)
names, training_images, training_labels = read_images( path_to_training_images, training_image_size)

model = cv2.face.EigenFaceRecognizer_create()
model.train(training_images, training_labels)

Run following block to recognize faces

In [22]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)
while cv2.waitKey(1) == -1 :
    success, frame = camera.read()
    if success:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2 )
            face_im = cv2.resize(gray[y:y+h, x:x+w], training_image_size )
            if face_im.size == 0:
                # The ROI is empty. Maybe the face is at the image edge.
                # Skip it.
                continue
            label, confidence = model.predict(face_im)

            text = '%s, confidence=%.2f' % (names[label], confidence)
            cv2.putText(frame, text, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('CapturingFaces',frame)
camera.release()
cv2.destroyWindow('CapturingFaces')